In [ ]:
import os
import cv2
import numpy as np
import copy
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import torch

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, LSTM, Reshape, TimeDistributed, Conv2D, MaxPooling2D

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.set_visible_devices([], 'GPU')

In [ ]:
def video_data_generator(directory, batch_size):
    target_size = (240, 240)
    class_names = sorted(os.listdir(directory))
    num_classes = len(class_names)
    
    while True:
        video_frames = []
        labels = []
        batch_count = 0
        
        for class_index, class_name in enumerate(class_names):
            class_dir = os.path.join(directory, class_name)
            for video_name in os.listdir(class_dir):
                video_path = os.path.join(class_dir, video_name)
                cap = cv2.VideoCapture(video_path)
                
                frames = []
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    cropped_frame = cv2.resize(frame, target_size)
                    frames.append(cropped_frame)
                
                cap.release()
                if len(frames) > 0:
                    video_frames.append(frames)
                    labels.append(class_index)
                
                if len(video_frames) == batch_size:
                    video_frames = np.array(video_frames)
                    video_frames = video_frames / 255.0
                    video_frames = np.reshape(video_frames, (1, -1, 240, 240, 3))
                    labels = tf.keras.utils.to_categorical(labels, num_classes, dtype=float)
                    yield video_frames, labels
                    video_frames = []
                    labels = []
                    batch_count += 1
                    
                    if batch_count == batch_size:
                        batch_count = 0
        
        if len(video_frames) > 0:
            l = len(video_frames)
            video_frames = np.array(video_frames)
            video_frames = video_frames / 255.0
            video_frames = np.reshape(video_frames, (1, -1, 240, 240, 3))
            labels = tf.keras.utils.to_categorical(labels, num_classes, dtype=float)
            yield video_frames, labels


In [ ]:
train_generator = video_data_generator("NewData/train", 1)
validation_generator = video_data_generator("NewData/validation", 1)

# video_frames, labels = next(train_generator)
# print(labels)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

steps_per_epoch = 600

In [ ]:
model = keras.Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(None,240,240,3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
# model = models.Sequential()
# model.add(LSTM(32, dropout=0.2, return_sequences=True, input_shape=(None, 240*240*3)))
# model.add(LSTM(64))
# # model.add(Flatten())
# model.add(Dense(512, activation='softmax'))
# model.add(Dropout(0.5))
# model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint("model-train-{epoch:01d}.h5")

In [ ]:
# model.load_weights('model-train-30.h5')

In [ ]:
history = model.fit(train_generator, steps_per_epoch = steps_per_epoch, epochs=30, validation_data=validation_generator,validation_steps=200, callbacks = [checkpoint])

In [ ]:
from matplotlib import pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
model.save("model.h5")

In [ ]:
test_generator = video_data_generator("NewData/test", 1)
model.evaluate(test_generator, steps=200)

In [ ]:
def preprocess_frames(frames_original):
    frames = []
    target_size = (240, 240)
    for frame in frames_original:
        if np.any(frame != 0):
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, threshold = cv2.threshold(gray_frame, 1, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if len(contours) > 0:
                x, y, w, h = cv2.boundingRect(contours[0])
                cropped_frame = frame[y:y+h, x:x+w, :]
                cropped_frame = cv2.resize(cropped_frame, target_size)
                frames.append(cropped_frame)
                
    frames = np.array(frames)
    frames = torch.tensor(frames, dtype=torch.float32)
    frames = frames / 255.0
    frames = np.reshape(frames, (-1, 240*240*3))
    return frames

In [ ]:
# video_path = 'data/test/Fight/6.mp4'
cap = cv2.VideoCapture(video_path)

frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

cap.release()

preprocessed_frames = preprocess_frames(frames)
preprocessed_frames = preprocessed_frames.reshape((1, -1, 240*240*3))
preprocessed_frames = tf.convert_to_tensor(preprocessed_frames, dtype=tf.float32) 

predictions = model.predict(preprocessed_frames)

In [ ]:
predicted_class = np.argmax(predictions)

In [ ]:
predicted_class